# Import dependencies

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import time

# Download data

## Set transforms

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.2, 0.2, 0.2]
    )
])

training_data = CIFAR10(
    root='data',
    train=True,
    transform=transform,
    download=True
)

test_data = CIFAR10(
    root='data',
    train=True,
    transform=transform,
    download=True
)

100%|████████████████████████| 170498071/170498071 [00:17<00:00, 9795203.34it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


# ResNet network

In [3]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# Training

## Hyperparameters

In [4]:
LEARNING_RATE = 1e-4
EPOCHS = 3
BATCH_SIZE = 4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

TRAIN_SPLIT = 0.75
VAR_SPLIT = 1 - TRAIN_SPLIT

TRAIN_SIZE = int(TRAIN_SPLIT * len(training_data))
VAL_SIZE = len(training_data) - TRAIN_SIZE

train_data, val_data = random_split(
    training_data,
    [TRAIN_SIZE, VAL_SIZE],
    torch.Generator().manual_seed(42)
)

## DataLoader

In [5]:
train_loader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_loader = DataLoader(
    dataset=val_data,
    batch_size=BATCH_SIZE,
)

test_loader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
)

## For plotting

In [6]:
H = {
    'avg_train_loss': [],
    'avg_val_loss': [],
    'train_acc': [],
    'val_acc': []
}
matplotlib.use('Agg')
train_steps = len(train_loader.dataset)
val_steps = len(val_loader.dataset)

## Model, loss function and optimizer

In [7]:
model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=0.001,
    momentum=0.9
)

## Train loop

In [8]:
print('[INFO] Start training ...')
start_time = time.time()

for epoch in range(EPOCHS):
    total_train_loss = 0
    total_val_loss = 0
    train_correct = 0
    val_correct = 0

    # train mode
    model.train()
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        # forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # back propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        train_correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

    avg_train_loss = total_train_loss / train_steps
    train_correct /= len(train_loader.dataset)

    # evaluation mode
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(val_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()
            val_correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

        avg_val_loss = total_val_loss / val_steps
        val_correct /= len(test_loader.dataset)

    H['avg_train_loss'].append(avg_train_loss)
    H['avg_val_loss'].append(avg_val_loss)
    H['train_acc'].append(train_correct)
    H['val_acc'].append(val_correct)

    print(f'[INFO] Epoch {epoch + 1}/{EPOCHS}')
    print(f'Train loss: {avg_train_loss}, train accuracy: {train_correct}')
    print(f'Val loss: {avg_val_loss}, val accuracy: {val_correct}')

end_time = time.time()
print(f'[INFO] Training and validating took {end_time - start_time}')

[INFO] Start training ...


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report

with torch.no_grad():
    model.eval()
    preds = []
    
    for batch_idx, (inputs, labels) in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        preds.extend(outputs.argmax(1).cpu().numpy())

    print(
        classification_report(
            test_data.targets.cpu().numpy(),
            np.array(preds),
            target_names=test_data.classes
        )
    )
        

In [ ]:
plt.style.use('ggplot')
plt.figure()
plt.plot(H['avg_train_loss'], label='avg_train_loss')
plt.plot(H['avg_val_loss'], label='avg_val_loss')
plt.title('Training loss and accuracy on dataset')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='lower left')
plt.savefig('./figures/Loss.png')

plt.figure()
plt.plot(H['train_acc'], label='train_accuracy')
plt.plot(H['val_acc'], label='val_accuracy')
plt.title('Training loss and accuracy on dataset')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower left')
plt.savefig('./figures/Accuracy.png')

torch.save(model, './model/test_model')